# Temporal CNN Model Evaluation and Predictions Visualization

In [1]:
#Script uses conda environment 'Python 3.8 (earthengine-gcloud)'

# Packages
import sys
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.plot import show
import datetime
import pprint
import time
import math
import random
from glob import glob
from functools import reduce
from pprint import pprint
import itertools
from scipy.spatial.distance import cdist

# Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Tensorflow & Keras
import tensorflow as tf
print(tf.__version__) 
import keras
from tensorflow.keras.utils import to_categorical


2.10.1


# Load Trained TemporalCNN Model

In [ ]:

# Load a trained model
model = keras.models.load_model(r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/temporalCNN/Archi7/TempCNN_100epochs_latlongenc_classweightinvfreq_nov202023-1.h5')

model.summary()


# Read and Format Testing Data from CSV

In [ ]:
# Load Testing Data

# Read testing set dataframe
test_df = pd.DataFrame(pd.read_csv("/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/datasets_oct23/test_spurge_landcover_landsat_spectral_allyears_69ksample_locations_MN.csv"))
test_df.head()
#print(test_df.shape)
nchannels = 7 #-- B G NDVI NIR Red SWIR1 SWIR2

# Select spectral data (first 63 columns)
batch_X = test_df.iloc[:, 1:64].to_numpy()
batch_X_spectral = batch_X.reshape(batch_X.shape[0],int(batch_X.shape[1]/nchannels),nchannels)
#print(batch_X_spectral[0])
#print(batch_X_spectral.shape)

# Select latitude/longitude coordinates (get last 3 columns, remove last 1 (class) column)
batch_X_coords = test_df.iloc[:, -3:].iloc[:, :-1].to_numpy()
#print(batch_X_coords.shape)

# Add X data(lat/long coordinates and spectral data) as list to X_test
# Add Y data(class) to Y_test
X_test = [batch_X_coords, batch_X_spectral]

# Select class data from last column
y_test = np.asarray(test_df.iloc[:, -1]).astype(int)


# Get unique class labels
unique_classes = np.unique(y_test)
print(unique_classes)
# Create a mapping dictionary
class_mapping = {class_label: index for index, class_label in enumerate(unique_classes)}
# Map the class indices to the new numeric indices
y_test_mapped = np.array([class_mapping[class_label] for class_label in y_test])

# Convert to one-hot encoding
y_test_one_hot = to_categorical(y_test_mapped, num_classes=len(unique_classes))

# Validate data are formatted correctly
# X_test[0] are lat/long coordinate pairs, (nsamples, 2)
# X_test[1] are 7 spectral bands across 9 timesteps (nsamples, 9, 7)
# y_test_one_hot are one-hot encoded class labels (nsamples, num_classes)
print(X_test[0].shape, X_test[1].shape, y_test_one_hot.shape)


# Evaluate Model Performance with Performance Metrics

In [ ]:
# Evaluate the model prediction

from sklearn.metrics import multilabel_confusion_matrix
from tabulate import tabulate

# Predict the model on withheld testing dataset
y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis=-1)
y_pred_flat = y_pred.flatten()
y_pred_flat = y_pred_flat.astype(int)

y_test = y_test.astype(int)    

# Reclassify y_test class values to match model predictions starting from 0
y_test[y_test==11] = 0 #Open Water
y_test[y_test==21] = 1 #Developed Open Space
y_test[y_test==22] = 2 #Developed Low Intensity
y_test[y_test==23] = 3 #Developed Medium Intensity
y_test[y_test==24] = 4 #Developed High Intensity
y_test[y_test==31] = 5 #Barren Land
y_test[y_test==41] = 6 #Deciduous Forest
y_test[y_test==42] = 7 #Evergreen Forest
y_test[y_test==43] = 8 #Mixed Forest
y_test[y_test==52] = 9 #Shrub/Scrub
y_test[y_test==71] = 10 #Grassland/Herbaceous
y_test[y_test==81] = 11 #Pasture/Hay
y_test[y_test==82] = 12 #Cultivated Crops
y_test[y_test==90] = 13 #Woody Wetlands
y_test[y_test==95] = 14 #Emergent Herbaceous Wetlands
y_test[y_test==99] = 15 #Leafy Spurge

y_test_flat = y_test.flatten()

#Count number of classes in the testing dataset
unique_vals, counts = np.unique(y_test, return_counts=True)

# Calculate confusion matrix
class_names = ["Open Water", "Developed Open Space", "Developed Low Intensity", "Developed Medium Intensity",
              "Developed High Intensity", "Barren Land", "Deciduous Forest", "Evergreen Forest", "Mixed Forest",
              "Shrub/Scrub", "Grassland/Herbaceous", "Pasture/Hay", "Cultivated Crops", "Woody Wetlands", "Emerg. Herb. Wetlands",
              "Leafy Spurge"]

class_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
c = multilabel_confusion_matrix(y_test_flat, y_pred_flat, labels = class_labels)
model_output_metrics = []
for i in range(len(class_labels)):
    tn=c[i, 0, 0]
    tp=c[i, 1, 1]
    fn=c[i, 1, 0]
    fp=c[i, 0, 1]
    accuracy = (tp+tn)/(tp+tn+fp+fn)
    TPR_Sens_Recall = tp/(tp+fn)
    TNR_Spec = tn/(tn+fp)
    FPR = fp/(fp+tn)
    FNR = fn/(fn+tp)
    precision = tp/(tp+fp)
    jaccard = tp/(tp+fp+fn)
    F05 = (1.25 * precision * TPR_Sens_Recall) / ((1.25 * precision) + TPR_Sens_Recall)
    F1 = (2 * precision * TPR_Sens_Recall) / (precision + TPR_Sens_Recall)
    F2 = (5 * precision * TPR_Sens_Recall) / ((5 * precision) + TPR_Sens_Recall)
    outputs = [class_names[i], counts[i], tp, tn, fp, fn, accuracy, TPR_Sens_Recall, TNR_Spec, FPR, FNR, precision, jaccard, F1]
    model_output_metrics.append(outputs)

# Print and format outputs
print(tabulate(model_output_metrics, floatfmt=".2f", headers=["Class Name", "Num Points", "TP", "TN", "FP", "FN", "Accuracy", "TPR/Sens/Recall", "TNR/Spec", "FPR", "FNR", "Precision", "Jaccard", "F1"]))

#Save tabulated results to file
#with open(res_file, 'w') as f:
#    f.write(tabulate(model_output_metrics, floatfmt=".2f", headers=["Class Name", "TP", "TN", "FP", "FN", "Accuracy", "TPR/Sens/Recall", "TNR/Spec", "FPR", "FNR", "Precision", "Jaccard", "F1"]))
    


# Plot Training History of Loss and Accuracy

In [ ]:
import matplotlib.pyplot as plt

# Read in Model Training History File (if Available)
model_history_path = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/temporalCNN/Archi6/trainingHistory-SB-latlong-spurgespatialthin001-blockcrossvalidation-noarchi6-norun-0.txt'

# Read data from the file
with open(model_history_path, 'r') as file:
    lines = file.readlines()

# Extract data columns
epochs = []
train_loss = []
val_loss = []
train_accuracy = []
val_accuracy = []

for line in lines:
    values = [float(x) for x in line.split()]
    epochs.append(values[-1])
    train_loss.append(values[0])
    val_loss.append(values[1])
    train_accuracy.append(values[2])
    val_accuracy.append(values[3])

# Plotting
plt.figure(figsize=(12, 6))

# Plotting Training Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_loss, label='Training Loss')
plt.plot(epochs, val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plotting Training Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, train_accuracy, label='Training Accuracy')
plt.plot(epochs, val_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


# Plot Confusion Matrix

In [ ]:

import os
import sys
from glob import glob
from tqdm import tqdm
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from skimage.io import imread, imshow, imsave
from sklearn.metrics import multilabel_confusion_matrix, confusion_matrix, cohen_kappa_score, accuracy_score, f1_score, precision_score, recall_score, jaccard_score, fbeta_score
from tensorflow.keras.models import load_model
from tabulate import tabulate

def plot_confusion_matrix(
        y_true,
        y_pred,
        classes,
        test_name,
        normalize=False,
        set_title=False,
        save_fig=False,
        cmap=plt.cm.Blues
):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    if set_title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    # and save it to log file
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
        #with open(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_log.txt', 'ab') as f:
        #    f.write(b'\nNormalized confusion matrix\n')
        #    np.savetxt(f, cm, fmt='%.3f')
    else:
        print('Confusion matrix, without normalization')
        #with open(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_log.txt', 'ab') as f:
        #    f.write(b'\nConfusion matrix, without normalization\n')
        #    np.savetxt(f, cm, fmt='%7u')

    #print(cm)
    #cm = cm[1:10]
    #cm = cm[:,1:]

    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    if normalize:
        im.set_clim(0., 1.)     # fixes missing '1.0' tick at top of colorbar
    cb = ax.figure.colorbar(im, ax=ax)
    if normalize:
        cb.set_ticks(np.arange(0., 1.2, 0.2))
        cb.set_ticklabels([f'{i/5:.1f}' for i in range(6)])
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title if set_title else None,
           ylabel='True label',
           xlabel='Predicted label')
    ax.set_ylim(len(cm)-0.5, -0.5)
    ax.xaxis.label.set_size(10)
    ax.yaxis.label.set_size(10)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if np.round(cm[i, j], 2) > 0.:
                ax.text(j, i, format(cm[i, j], fmt),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
            else:
                ax.text(j, i, '–',
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    if save_fig:
        if normalize:
            plt.savefig(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_cm_normal.pdf')
        else:
            plt.savefig(f'F:/PlanetScope_LSTM_Imagery/reports/logs_and_plots/{test_name}_cm_non_normal.pdf')
    return fig, ax, cm




inv_category_dict = {0:"Open Water", 1:"Developed Open Space", 2:"Developed Low Intensity", 3:"Developed Medium Intensity", 4:"Developed High Intensity", 5:"Barren Land", 
                     6:"Deciduous Forest", 7:"Evergreen Forest", 8:"Mixed Forest", 9:"Shrub/Scrub", 10:"Grassland/Herbaceous", 11:"Pasture/Hay", 
                     12:"Cultivated Crops", 13:"Woody Wetlands", 14:"Emerg. Herb. Wetlands", 15:"Leafy Spurge"}

class_names = [inv_category_dict[i] for i in np.arange(0, 15)]

# Calculate confusion matrix
class_names_list = ["Open Water", "Developed Open Space", "Developed Low Intensity", "Developed Medium Intensity",
             "Developed High Intensity", "Barren Land", "Deciduous Forest", "Evergreen Forest", "Mixed Forest",
             "Shrub/Scrub", "Grassland/Herbaceous", "Pasture/Hay", "Cultivated Crops", "Woody Wetlands", "Emerg. Herb. Wetlands",
             "Leafy Spurge"]

#class_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# save plot of normalized cm
cm = plot_confusion_matrix(
    y_test_flat,
    y_pred_flat,
    classes=class_names_list,
    test_name="myModel",
    normalize=True,
    save_fig=False
)

conf_df = pd.DataFrame(cm[2], columns = class_names_list)

#np.savetxt(conf_file, conf_df, fmt='%6f')


In [ ]:
print(cm)

# Define Model Prediction Mosaic Rasters

In [ ]:
# Visualize Model Predictions of Leafy Spurge
# To Generate .TIF Predictions, Run Inference on Landsat Tiles in r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles'
# Using slurm to submit predictRasters-spurgesoftmax-latlon-encoding.py Python script that generates predictions, and gdal_merge tool to mosaic predictions

# Prediction from Landsat imagery between 2000 - 2002
# Argmax contains only Leafy Spurge class, data range from 0-1
prediction_2001_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2001/merged_predictions_argmax_landsat_temporalcnn_2001_dec23.tif'
# Softmax contains all class values predicted from model as integers [0-15]
prediction_2001_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2001/merged_predictions_softmax_landsat_temporalcnn_2001_dec23.tif'


# Prediction from Landsat imagery between 2003 - 2005
prediction_2004_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2004/merged_predictions_argmax_landsat_temporalcnn_2004_dec23.tif'
prediction_2004_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2004/merged_predictions_softmax_landsat_temporalcnn_2004_dec23.tif'


# Prediction from Landsat imagery between 2006 - 2008
prediction_2007_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2007/merged_predictions_argmax_landsat_temporalcnn_2007_dec23.tif'
prediction_2007_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2007/merged_predictions_softmax_landsat_temporalcnn_2007_dec23.tif'


# Prediction from Landsat imagery between 2009 - 2011
prediction_2010_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2010/merged_predictions_argmax_landsat_temporalcnn_2010_dec23.tif'
prediction_2010_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2010/merged_predictions_softmax_landsat_temporalcnn_2010_dec23.tif'


# Prediction from Landsat imagery between 2013 - 2015
prediction_2014_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2014/merged_predictions_argmax_landsat_temporalcnn_2014_dec23.tif'
prediction_2014_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2014/merged_predictions_softmax_landsat_temporalcnn_2014_dec23.tif'


# Prediction from Landsat imagery between 2015 - 2017
prediction_2016_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2016/merged_predictions_argmax_landsat_temporalcnn_2016_dec23.tif'
prediction_2016_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2016/merged_predictions_softmax_landsat_temporalcnn_2016_dec23.tif'


# Prediction from Landsat imagery between 2018 - 2020
prediction_2019_argmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2019/merged_predictions_argmax_landsat_temporalcnn_2019_dec23.tif'
prediction_2019_softmax = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2019/merged_predictions_softmax_landsat_temporalcnn_2019_dec23.tif'



# Plot one prediction mosaic raster for leafy spurge (probability values)

In [ ]:
# Plot Predicted Probability of Leafy Spurge Raster

# Choose one of the raster files to plot
selected_raster = prediction_2004_argmax  # Replace with the desired file path

# Read and plot the selected raster
with rasterio.open(selected_raster) as src:
    data = src.read(1, masked=True)
    data.shape
    plt.figure(figsize=(30, 30))
    show(data, transform=src.transform, cmap='viridis', title=f"{selected_raster}")
    plt.axis('off')
    plt.show()
    


# Plot one prediction mosaic raster for leafy spurge (classification values)

In [ ]:
# Plot Land Cover CLasses Raster

# Choose one of the raster files to plot
selected_raster = prediction_2001_softmax  # Replace with the desired file path

# Read and plot the selected raster
with rasterio.open(selected_raster) as src:
    data = src.read(1, masked=True)
    plt.figure(figsize=(30, 30))
    show(data, transform=src.transform, cmap='viridis', title=f"{selected_raster}")
    plt.axis('off')
    plt.show()
    

# Evaluate Model Performance Based on Leafy Spurge from 2021 - 2023

In [ ]:
# Read in Leafy Spurge occurrence records from 2021, 2022, and 2023

spurge_data = r'/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/datasets_oct23/leafy-spurge-csv/eddmaps_inaturalist_spurge_observations_filtered_combined_october2023_minnesota.csv'

# Read in .CSV file
spurge_df = pd.read_csv(spurge_data)

# Extract latitude, longitude, and ObsDate columns from the DataFrame
latitude = spurge_df['Latitude']
longitude = spurge_df['Longitude']
obs_date = pd.to_datetime(spurge_df['ObsDate'], errors='coerce')  # Convert 'ObsDate' to datetime

# Select only records where 'ObsDate' contains the years 2021 to 2023
selected_records = spurge_df[(obs_date >= '2021-01-01') & (obs_date <= '2023-12-31')]

selected_records_old = spurge_df[obs_date <= '2021-01-01']

# Subset records by dates used to develop models
selected_records_2000_2002 = spurge_df[(obs_date >= '2000-01-01') & (obs_date <= '2002-12-31')]

selected_records_2003_2005 = spurge_df[(obs_date >= '2003-01-01') & (obs_date <= '2005-12-31')]

selected_records_2006_2008 = spurge_df[(obs_date >= '2006-01-01') & (obs_date <= '2008-12-31')]

selected_records_2009_2011 = spurge_df[(obs_date >= '2009-01-01') & (obs_date <= '2011-12-31')]

selected_records_2013_2015 = spurge_df[(obs_date >= '2013-01-01') & (obs_date <= '2015-12-31')]

selected_records_2015_2017 = spurge_df[(obs_date >= '2015-01-01') & (obs_date <= '2017-12-31')]

selected_records_2018_2020 = spurge_df[(obs_date >= '2018-01-01') & (obs_date <= '2020-12-31')]

# Read in the shapefile
shapefile_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/shapefiles/Minnesota_Shapefile.shp' 
shapefile = gpd.read_file(shapefile_path)

# Plot the shapefile
shapefile.plot(color='gray', figsize=(20, 12))

# Plot all records in blue
plt.scatter(longitude, latitude, color='blue', alpha=0.6, edgecolors='w', s=50, label='Other Records')  # All records in blue

# Plot selected records in red
plt.scatter(selected_records['Longitude'], selected_records['Latitude'], color='red', alpha=0.6, edgecolors='w', s=50, label='2021-2023 Records')  # Selected records in red

# Add legend
plt.legend()

# Add labels and title
plt.xlabel('Longitude', fontsize=20)
plt.ylabel('Latitude', fontsize=20)
plt.title('Leafy Spurge Occurrences', fontsize=20)

# Show the plot
plt.grid(True)
plt.show()


In [ ]:
selected_records

# Extract Predicted Leafy Spurge Values (0-1) from 2019 Raster using the 2021 - 2023 Leafy Spurge Occurrences

- How well are leafy spurge occurrences from 2021-2023 predicted by the 2019 Landsat imagery?

In [ ]:

# Read the raster file
raster_file_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2019/merged_predictions_argmax_landsat_temporalcnn_2019_dec23.tif'
with rasterio.open(raster_file_path) as src:
    # Extract raster values at points
    #selected_values_newspurge = [x[0] for x in src.sample(zip(selected_records['Longitude'], selected_records['Latitude']))]
    selected_values = [x[0] for x in src.sample(zip(selected_records['Longitude'], selected_records['Latitude']))]

    
# Create stacked bar plots
plt.figure(figsize=(16, 12))
plt.hist(selected_values, bins=50, color=['red'], alpha=0.7, label=['Leafy Spurge (2021 - 2023)'], stacked=True)
plt.xlabel('Predicted Probability of Leafy Spurge', fontsize=18)  # Increase font size for x-axis label
plt.ylabel('Number of Records', fontsize=18)  # Increase font size for y-axis label
plt.title('Bar Plot of Probability Values for Leafy Spurge Records', fontsize=18)  # Increase font size for title
plt.xticks(fontsize=18)  # Increase font size for x-axis ticks
plt.yticks(fontsize=18)  # Increase font size for y-axis ticks
plt.legend(fontsize=18)  # Add legend with increased font size
plt.grid(True)
plt.show()



# Extract Predicted Class Values (0-16) from 2019 Raster using the 2021 - 2023 Leafy Spurge Occurrences

- How well does the model classify leafy spurge occurrences from 2021 - 2023 given the prediction from the 2019 Landsat imagery?

In [ ]:
# Read the raster file
raster_file_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2019/merged_predictions_softmax_landsat_temporalcnn_2019_dec23.tif'
with rasterio.open(raster_file_path) as src:
    # Extract raster values at points
    #selected_values_softmax = [x[0] for x in src.sample(zip(selected_records['Longitude'], selected_records['Latitude']))]
    selected_values = [x[0] for x in src.sample(zip(selected_records['Longitude'], selected_records['Latitude']))]
    

class_names = ["Open Water", "Developed Open Space", "Developed Low Intensity", "Developed Medium Intensity",
             "Developed High Intensity", "Barren Land", "Deciduous Forest", "Evergreen Forest", "Mixed Forest",
             "Shrub/Scrub", "Grassland/Herbaceous", "Pasture/Hay", "Cultivated Crops", "Woody Wetlands", "Emerg. Herb. Wetlands",
             "Leafy Spurge"]
    
# Create a bar plot of each category
plt.figure(figsize=(12, 6))
plt.bar(range(16), [selected_values.count(i) for i in range(16)], color='red', alpha=0.7)
plt.xticks(range(16), class_names, rotation=45, ha='right', fontsize=12)  # Increase font size for x-axis labels
plt.yticks(fontsize=12)  # Increase font size for y-axis labels
plt.xlabel('Land Cover Class', fontsize=18)  # Increase font size for x-axis label
plt.ylabel('Frequency', fontsize=18)  # Increase font size for y-axis label
plt.title('Bar Plot of Land Cover Classes for 2018 - 2020 Records', fontsize=16)  # Increase font size for title
plt.grid(True)
plt.show()
    

In [ ]:
# Read the raster file
raster_file_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/minnesota_landsat_tiles/predictions_2019/merged_predictions_softmax_landsat_temporalcnn_2019_dec23.tif'
with rasterio.open(raster_file_path) as src:
    # Extract raster values at points
    selected_values_softmax = [x[0] for x in src.sample(zip(selected_records['Longitude'], selected_records['Latitude']))]
    selected_values_oldsoftmax = [x[0] for x in src.sample(zip(selected_records_old['Longitude'], selected_records_old['Latitude']))]

# Create a stacked bar plot for each category
plt.figure(figsize=(12, 6))
plt.bar(range(16), [selected_values_softmax.count(i) + selected_values_oldsoftmax.count(i) for i in range(16)], color='red', alpha=0.7, label='Leafy Spurge (2021 - 2023)')  # Blue bars for new softmax
plt.bar(range(16), [selected_values_oldsoftmax.count(i) for i in range(16)], bottom=[selected_values_softmax.count(i) for i in range(16)], color='blue', alpha=0.7, label='Leafy Spurge (2000 - 2020)')  # Red bars for old softmax
plt.xticks(range(16), class_names, rotation=45, ha='right', fontsize=12)  # Increase font size for x-axis labels
plt.yticks(fontsize=12)  # Increase font size for y-axis labels
plt.xlabel('Land Cover Class', fontsize=18)  # Increase font size for x-axis label
plt.ylabel('Frequency', fontsize=18)  # Increase font size for y-axis label
plt.title('Predicted Land Cover Classes given Leafy Spurge Records', fontsize=16)  # Increase font size for title
plt.legend(fontsize=16)  # Add legend with increased font size
plt.grid(True)
plt.show()


# Extract Predicted Leafy Spurge Values (0-1) from 2019 Raster using the Leafy Spurge Polygons from 2019 - 2023 in Twin Cities Region

- How well are leafy spurge populations from 2019-2023 predicted by the 2019 Landsat imagery?
- Does probability of detection relate to population size (from EddMaps polygons)?


# Evaluate predicted probability of leafy spurge with surveyed EddMaps polygons covering Twin Cities

In [ ]:
from shapely.geometry import box

# Read in the leafy spurge polygons shapefile
spurge_shapefile_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/shapefiles/leafy-spurge-eddmaps-polygons-dec23.shp' 
spurge_shapefile = gpd.read_file(spurge_shapefile_path)

# Convert 'ObsDate' to datetime
spurge_shapefile['ObsDate'] = pd.to_datetime(spurge_shapefile['ObsDate'], errors='coerce')

# Filter polygons based on conditions
filtered_spurge_shapefile = spurge_shapefile[
    (spurge_shapefile['ObsDate'].dt.year.isin([2018, 2019, 2020, 2021, 2022, 2023])) &
    (spurge_shapefile['Status'] == 'Positive') &
    (spurge_shapefile['OccStatus'] == 'Detected') &
    (~spurge_shapefile['InfestAcre'].isnull()) &
    (spurge_shapefile['InfestAcre'] > 0)
]

# Define the bounding box coordinates
bbox = box(minx=-93.8498, miny=44.5495, maxx=-92.8248, maxy=45.2814)

# Apply spatial filter
cropped_spurge_shapefile = filtered_spurge_shapefile.cx[bbox.bounds[0]:bbox.bounds[2], bbox.bounds[1]:bbox.bounds[3]]

# Print summary information
print(f"Original shapefile has {len(spurge_shapefile)} polygons.")
print(f"After filtering with date, status, and area, the shapefile has {len(filtered_spurge_shapefile)} polygons.")
print(f"After filtering with spatial location, the shapefile has {len(cropped_spurge_shapefile)} polygons.")



# Evaluate predicted probability of leafy spurge with surveyed EddMaps polygons covering entire state

In [ ]:
# Alternatively, crop the leafy spurge polygons by the entire state of Minnesota
from geopandas.tools import overlay

# Read in the shapefile representing the state boundaries of Minnesota
state_shapefile_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/shapefiles/Minnesota_Shapefile.shp'
state_shapefile = gpd.read_file(state_shapefile_path)

# Read in the leafy spurge polygons shapefile
spurge_shapefile_path = '/panfs/jay/groups/31/moeller/shared/leafy-spurge-demography/shapefiles/leafy-spurge-eddmaps-polygons-dec23.shp' 
spurge_shapefile = gpd.read_file(spurge_shapefile_path)

# Convert 'ObsDate' to datetime
spurge_shapefile['ObsDate'] = pd.to_datetime(spurge_shapefile['ObsDate'], errors='coerce')

# Filter polygons based on conditions
filtered_spurge_shapefile = spurge_shapefile[
    (spurge_shapefile['ObsDate'].dt.year.isin([2018, 2019, 2020, 2021, 2022, 2023])) &
    (spurge_shapefile['Status'] == 'Positive') &
    (spurge_shapefile['OccStatus'] == 'Detected') &
    (~spurge_shapefile['InfestAcre'].isnull()) &
    (spurge_shapefile['InfestAcre'] > 0)
]

# Perform overlay to crop spurge_shapefile by state_shapefile
cropped_spurge_shapefile = overlay(filtered_spurge_shapefile, state_shapefile, how='intersection')

# Print summary information
print(f"Original shapefile has {len(spurge_shapefile)} polygons.")
print(f"After filtering with date, status, and area, the shapefile has {len(filtered_spurge_shapefile)} polygons.")
print(f"After cropping by the state boundaries, the shapefile has {len(cropped_spurge_shapefile)} polygons.")



In [ ]:
# Create a new column 'ObsYear' to store the year of observation
cropped_spurge_shapefile['ObsYear'] = cropped_spurge_shapefile['ObsDate'].dt.year

# Count the number of observations per year
observations_per_year = cropped_spurge_shapefile.groupby('ObsYear').size()

print(observations_per_year)

# Plot a bar plot
plt.figure(figsize=(10, 6))
observations_per_year.plot(kind='bar', color='blue', alpha=0.7)
plt.xlabel('Year')
plt.ylabel('Number of Observations')
plt.title('Number of Observations per Year')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.show()

In [ ]:
cropped_spurge_shapefile

In [ ]:

# Create a new column 'InfestSquareMeters' and convert 'InfestAcre' to square meters
cropped_spurge_shapefile['InfestSquareMeters'] = cropped_spurge_shapefile['InfestAcre'] * 4046.86  # 1 acre = 4046.86 square meters

# Log-transform 'InfestSquareMeters'
cropped_spurge_shapefile['LogInfestSquareMeters'] = np.log1p(cropped_spurge_shapefile['InfestSquareMeters'])


# Plot a histogram of the 'InfestAcre' field
plt.figure(figsize=(10, 6))
plt.hist(cropped_spurge_shapefile['InfestSquareMeters'], bins=10000, color='blue', alpha=0.7)
plt.xlabel('Square Meters', fontsize=18)
plt.ylabel('Frequency', fontsize=18)
plt.title('Histogram of Infested Square Meters', fontsize=18)
plt.grid(True)
plt.xlim(0, 2000)
plt.show()


np.min(cropped_spurge_shapefile['InfestSquareMeters'])
np.mean(cropped_spurge_shapefile['InfestSquareMeters'])
np.max(cropped_spurge_shapefile['InfestSquareMeters'])



# Extract Predicted Leafy Spurge Probability from Each Polygon of Survyed Leafy Spurge From 2018 - 2023

In [ ]:
from rasterstats import zonal_stats

# Use zonal_stats to extract raster values for each polygon
zs_result = zonal_stats(cropped_spurge_shapefile, prediction_2019_argmax, stats='max', geojson_out=True)

# Create a new GeoDataFrame from the zonal_stats result
result_df = gpd.GeoDataFrame.from_features(zs_result, crs=cropped_spurge_shapefile.crs)



In [ ]:
result_df

In [ ]:
# Given values
pixel_resolution = 30

# Calculate the number of pixels
result_df['NumPixels'] = result_df['InfestSquareMeters'] / (pixel_resolution ** 2)

# Plot a scatter plot with 'Number of Pixels'
plt.figure(figsize=(20, 12))

# Scatter plot with larger points
plt.scatter(result_df['NumPixels'], result_df['max'], alpha=0.6, edgecolors='w', s=100, c='blue')

# Increase font size for axis labels and title
plt.xlabel('Leafy Spurge Population Size (Num Landsat 30x30m Px)', fontsize=30)
plt.ylabel('Mean Predicted Value', fontsize=30)
plt.title('Scatter Plot Leafy Spurge Prediction vs. Population Size', fontsize=30)

# Increase font size for tick labels
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)

plt.grid(True)

# Adjust the x-axis limit as needed
plt.xlim(0, 25)

plt.show()


In [ ]:
# Given values
pixel_resolution = 30

# Calculate the number of pixels
result_df['NumPixels'] = result_df['InfestSquareMeters'] / (pixel_resolution ** 2)

# Define the bins for categorization
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, np.inf]  # Adjust the bin edges as needed

# Create a new categorical column 'NumPixelsCategory' based on the bins
result_df['NumPixelsCategory'] = pd.cut(result_df['NumPixels'], bins=bins, labels=['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>10'])

# Plot a scatter plot with categorical x-axis
plt.figure(figsize=(12, 6))
scatter = plt.scatter(result_df['NumPixels'], result_df['max'], alpha=0.6, edgecolors='w', s=50, c=result_df['NumPixelsCategory'].cat.codes, cmap='viridis')
plt.xlabel('Number of Pixels')
plt.ylabel('Average Prediction (Argmax)')
plt.title('Scatter Plot of Number of Pixels vs. Average Prediction')
plt.grid(True)

# Add a legend for the categories
legend = plt.legend(*scatter.legend_elements(), title='NumPixelsCategory')
plt.gca().add_artist(legend)

# Adjust the x-axis limit as needed
plt.xlim(0,25)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Given values
pixel_resolution = 30

# Calculate the number of pixels
result_df['NumPixels'] = result_df['InfestSquareMeters'] / (pixel_resolution ** 2)

# Define the bins for categorization
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, np.inf]  # Adjust the bin edges as needed

# Create a new categorical column 'NumPixelsCategory' based on the bins
result_df['NumPixelsCategory'] = pd.cut(result_df['NumPixels'], bins=bins, labels=['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>10'])

# Convert 'NumPixelsCategory' to categorical with a custom order
category_order = ['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '>10']
result_df['NumPixelsCategory'] = pd.Categorical(result_df['NumPixelsCategory'], categories=category_order, ordered=True)

# Create boxplots for each category
plt.figure(figsize=(18, 10))

# Larger elements in the boxplot
sns.boxplot(x='NumPixelsCategory', y='max', data=result_df, width=0.7, linewidth=2)

# Increase font size for axis labels and title
plt.xlabel('Leafy Spurge Population Size (Num Landsat 30x30m Px)', fontsize=30)
plt.ylabel('Max Predicted Value', fontsize=30)
plt.title('Boxplots of Max Prediction for Number of Pixels Categories', fontsize=30)

# Increase font size for tick labels
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)

plt.grid(True)
plt.show()
